In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_x': 'Distance_x',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions': 'carbon_emissions'
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_x) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_x'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions': row['carbon_emissions']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """Optimize to minimize carbon emissions while considering demand and TOPSIS score."""
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_emissions += num_vehicles * vehicle['carbon_emissions']
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        return -total_emissions + weighted_topsis - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to minimize carbon emissions."""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                next_generation.extend([parent1, parent2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(num_vehicles * vehicle_data['carbon_emissions'], 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        return pd.DataFrame(results)

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "data/topsis_results_2023.csv"
    results_df = main(csv_path)
    print(results_df)

              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel          69347.98          B20   
2   Size S1, Distance D1     LNG         125913.45          LNG   
3   Size S1, Distance D2  Diesel         208043.93          B20   
4   Size S1, Distance D2     LNG         125913.45          LNG   
5   Size S1, Distance D3  Diesel         485435.83          B20   
6   Size S1, Distance D4     LNG         125913.45          LNG   
7   Size S2, Distance D1     BEV              0.00  Electricity   
8   Size S2, Distance D1     LNG         217732.24          LNG   
9   Size S2, Distance D2  Diesel          72337.66          B20   
10  Size S2, Distance D2     LNG         348371.59          LNG   
11  Size S2, Distance D3     LNG         174185.79          LNG   
12  Size S2, Distance D4     LNG          43546.45          LNG   
13  Size S3, Distance D1  Diesel         249343.14          B2

In [ ]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_x': 'Distance_x',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions': 'carbon_emissions'
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_x) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_x'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions': row['carbon_emissions']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """Optimize to minimize carbon emissions while considering demand and TOPSIS score."""
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_emissions += num_vehicles * vehicle['carbon_emissions']
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        return -total_emissions + weighted_topsis - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to minimize carbon emissions."""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                next_generation.extend([parent1, parent2])
            
            population = next_generation[:len(population)]
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(num_vehicles * vehicle_data['carbon_emissions'], 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        return pd.DataFrame(results)

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "data/topsis_results_2023.csv"
    results_df = main(csv_path)
    # print(results_df)
    results_df.to_csv("optimized_fleet_Carbonmin.csv", index=False)


              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel          69347.98          B20   
2   Size S1, Distance D1     LNG          83942.30          LNG   
3   Size S1, Distance D2  Diesel         346739.88          B20   
4   Size S1, Distance D2     LNG          41971.15          LNG   
5   Size S1, Distance D3  Diesel         208043.93          B20   
6   Size S1, Distance D3     LNG         167884.59          LNG   
7   Size S1, Distance D4     LNG         125913.45          LNG   
8   Size S2, Distance D1     BEV              0.00  Electricity   
9   Size S2, Distance D1  Diesel          72337.66          B20   
10  Size S2, Distance D1     LNG         130639.34          LNG   
11  Size S2, Distance D2     LNG         348371.59          LNG   
12  Size S2, Distance D3  Diesel          72337.66          B20   
13  Size S2, Distance D3     LNG         130639.34          LN

In [2]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_x': 'Distance_x',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions': 'carbon_emissions'
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')

    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_x) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_x'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions': row['carbon_emissions']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        """Check if the total number of vehicles in the solution is within the dynamic maximum limit"""
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        """Generate initial population of fleet combinations"""
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population

    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        """Optimize to minimize carbon emissions while considering demand and TOPSIS score."""
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        total_capacity = 0
        weighted_topsis = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_emissions += num_vehicles * vehicle['carbon_emissions']
            total_capacity += num_vehicles * vehicle['yearly_range']
            weighted_topsis += num_vehicles * vehicle['topsis_score']
        
        demand = vehicles[0]['demand']
        demand_penalty = max(0, demand - total_capacity) * 1000
        
        return -total_emissions + weighted_topsis - demand_penalty

    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        """Run genetic algorithm to minimize carbon emissions."""
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                next_generation.extend([parent1, parent2])
            
            population = next_generation[:len(population)]
        
        return best_solution


    def evaluate_solution(self, results_df: pd.DataFrame) -> pd.DataFrame:
        """Evaluate the quality of the optimized results."""
        results_df['Utilization'] = results_df['no_of_vehicles'] / results_df['Max Vehicles']
        results_df['Demand Fulfillment'] = results_df['Yearly Range'] * results_df['no_of_vehicles'] / results_df['Demand']
        results_df['Efficiency Score'] = results_df['Demand Fulfillment'] / (results_df['Carbon Emissions'] + 1)
        return results_df

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(num_vehicles * vehicle_data['carbon_emissions'], 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        return pd.DataFrame(results)


def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()  # Now, this method exists
    evaluated_results = optimizer.evaluate_solution(optimized_results)
    print(evaluated_results)
    return evaluated_results

if __name__ == "__main__":
    csv_path = "data/topsis_results_2023.csv"
    results_df = main(csv_path)
    



              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel          69347.98          B20   
2   Size S1, Distance D1     LNG         125913.45          LNG   
3   Size S1, Distance D2  Diesel         277391.90          B20   
4   Size S1, Distance D2     LNG         167884.59          LNG   
5   Size S1, Distance D3  Diesel         624131.78          B20   
6   Size S1, Distance D4     LNG         125913.45          LNG   
7   Size S2, Distance D1     BEV              0.00  Electricity   
8   Size S2, Distance D1     LNG         261278.69          LNG   
9   Size S2, Distance D2  Diesel         289350.63          B20   
10  Size S2, Distance D2     LNG         130639.34          LNG   
11  Size S2, Distance D3     LNG         174185.79          LNG   
12  Size S2, Distance D4     LNG          43546.45          LNG   
13  Size S3, Distance D1     BEV              0.00  Electricit

In [4]:
results_df.to_csv("optimized_fleet_Carbonmin.csv", index=False)


In [5]:
def main(csv_2023_results: str, csv_2024: str, output_2024: str):
    # Load 2023 optimized results from saved CSV
    optimized_results_2023 = pd.read_csv(csv_2023_results)

    print("\n--- Loaded Optimized Fleet for 2023 ---")
    print(optimized_results_2023.head())

    # Load 2024 data
    data_df_2024 = load_and_preprocess_data(csv_2024)
    optimizer_2024 = FleetOptimizer(data_df_2024)

    # Integrate 2023 vehicles into 2024 optimization
    def modify_2024_fleet_with_2023(optimizer_2024, optimized_results_2023):
        for _, row in optimized_results_2023.iterrows():
            size_distance = (row['Allocation'].split()[1], row['Allocation'].split()[3])  # Extract (Size, Distance)
            vehicle_type = row['Vehicle']
            previous_allocation = row['no_of_vehicles']

            if size_distance in optimizer_2024.vehicles_by_size_distance:
                for vehicle in optimizer_2024.vehicles_by_size_distance[size_distance]:
                    if vehicle['vehicle_type'] == vehicle_type:
                        vehicle['previous_allocation'] = previous_allocation  # Store 2023 allocation

    modify_2024_fleet_with_2023(optimizer_2024, optimized_results_2023)

    # Optimize fleet for 2024
    optimized_results_2024 = optimizer_2024.get_optimized_results()
    evaluated_results_2024 = optimizer_2024.evaluate_solution(optimized_results_2024)

    print("\n--- Optimized Fleet for 2024 ---")
    print(evaluated_results_2024)

    # Save 2024 results
    evaluated_results_2024.to_csv(output_2024, index=False)
    print(f"\n--- Optimized 2024 fleet saved to {output_2024} ---")

    return evaluated_results_2024


if __name__ == "__main__":
    csv_2023_results = "optimized_fleet_Carbonmin.csv"  # Pre-saved optimized 2023 results
    csv_2024 = "data/topsis_results_2024.csv"  # New 2024 dataset
    output_2024 = "optimized_fleet_Carbonmin_2024.csv"  # Save optimized 2024 results

    results_2024 = main(csv_2023_results, csv_2024, output_2024)



--- Loaded Optimized Fleet for 2023 ---
             Allocation Vehicle  Carbon Emissions         Fuel  \
0  Size S1, Distance D1     BEV              0.00  Electricity   
1  Size S1, Distance D1  Diesel          69347.98          B20   
2  Size S1, Distance D1     LNG         125913.45          LNG   
3  Size S1, Distance D2  Diesel         277391.90          B20   
4  Size S1, Distance D2     LNG         167884.59          LNG   

   no_of_vehicles  Max Vehicles   Demand  Yearly Range  Utilization  \
0               1             9   869181        102000     0.111111   
1               1             9   869181        102000     0.111111   
2               3             9   869181        102000     0.333333   
3               4            26  2597094        102000     0.153846   
4               4            26  2597094        102000     0.153846   

   Demand Fulfillment  Efficiency Score  
0            0.117352      1.173519e-01  
1            0.117352      1.692193e-06  
2        

KeyError: 'Fuel_Costs'

In [3]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def optimize(self, size_distance: Tuple) -> Dict:
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        solution = {v['vehicle_type']: 0 for v in vehicles}
        remaining_vehicles = max_vehicles
        vehicle_types = list(solution.keys())
        
        while remaining_vehicles > 0 and vehicle_types:
            vehicle_type = random.choice(vehicle_types)
            solution[vehicle_type] += 1
            remaining_vehicles -= 1
        
        return solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km']
                    total_carbon_emissions += emissions
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        results_df = pd.DataFrame(results)
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


Total Carbon Emissions: 97.47092099106001 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel              0.68          B20   
2   Size S1, Distance D1     LNG              1.65          LNG   
3   Size S1, Distance D2  Diesel              9.52          B20   
4   Size S1, Distance D2     LNG              4.94          LNG   
5   Size S1, Distance D3  Diesel             10.20          B20   
6   Size S1, Distance D3     LNG              7.41          LNG   
7   Size S1, Distance D4  Diesel              2.04          B20   
8   Size S1, Distance D4     LNG              0.82          LNG   
9   Size S2, Distance D1     BEV              0.00  Electricity   
10  Size S2, Distance D1  Diesel              3.41          B20   
11  Size S2, Distance D1     LNG              1.23          LNG   
12  Size S2, Distance D2  Diesel              5.46          B20   
13

In [5]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def optimize(self, size_distance: Tuple) -> Dict:
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        solution = {v['vehicle_type']: 0 for v in vehicles}
        remaining_vehicles = max_vehicles
        vehicle_types = list(solution.keys())
        
        while remaining_vehicles > 0 and vehicle_types:
            vehicle_type = random.choice(vehicle_types)
            solution[vehicle_type] += 1
            remaining_vehicles -= 1
        
        return solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        results_df = pd.DataFrame(results)
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


Total Carbon Emissions: 18525690.957629822 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel         590940.62          B20   
2   Size S1, Distance D1     LNG         357652.20          LNG   
3   Size S1, Distance D2  Diesel        1765717.77          B20   
4   Size S1, Distance D2     LNG        1068657.04          LNG   
5   Size S1, Distance D3  Diesel        2238179.41          B20   
6   Size S1, Distance D3     LNG        1354602.77          LNG   
7   Size S1, Distance D4  Diesel         281685.36          B20   
8   Size S1, Distance D4     LNG         170483.10          LNG   
9   Size S2, Distance D1     BEV              0.00  Electricity   
10  Size S2, Distance D1  Diesel         679492.18          B20   
11  Size S2, Distance D1     LNG         409046.58          LNG   
12  Size S2, Distance D2  Diesel         943935.46          B20   
1

Total Carbon Emissions: 18220852.976719797 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D1  Diesel         590940.62          B20   
2   Size S1, Distance D2  Diesel        1765717.77          B20   
3   Size S1, Distance D2     LNG        1068657.04          LNG   
4   Size S1, Distance D3  Diesel        2238179.41          B20   
5   Size S1, Distance D3     LNG        1354602.77          LNG   
6   Size S1, Distance D4  Diesel         281685.36          B20   
7   Size S1, Distance D4     LNG         170483.10          LNG   
8   Size S2, Distance D1     BEV              0.00  Electricity   
9   Size S2, Distance D1  Diesel         679492.18          B20   
10  Size S2, Distance D1     LNG         409046.58          LNG   
11  Size S2, Distance D2  Diesel         943935.46          B20   
12  Size S2, Distance D2     LNG         568238.42          LNG   
1

In [8]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def optimize(self, size_distance: Tuple, remaining_carbon: float) -> Dict:
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        solution = {v['vehicle_type']: 0 for v in vehicles}
        remaining_vehicles = max_vehicles
        vehicle_types = sorted(vehicles, key=lambda v: v['carbon_emissions_per_km'])
        
        for vehicle in vehicle_types:
            if remaining_vehicles > 0:
                no_of_km = vehicle['demand'] / remaining_vehicles
                emissions_per_vehicle = vehicle['carbon_emissions_per_km'] * no_of_km
                max_allowed_vehicles = min(remaining_vehicles, remaining_carbon // emissions_per_vehicle)
                
                solution[vehicle['vehicle_type']] = int(max_allowed_vehicles)
                remaining_vehicles -= int(max_allowed_vehicles)
                remaining_carbon -= int(max_allowed_vehicles) * emissions_per_vehicle
                
                if remaining_carbon <= 0:
                    break
        
        return solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        remaining_carbon = self.CARBON_LIMIT
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance, remaining_carbon)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions
                    remaining_carbon -= emissions
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        results_df = pd.DataFrame(results)
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


ZeroDivisionError: float floor division by zero

In [2]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def optimize(self, size_distance: Tuple, remaining_carbon: float) -> Dict:
        vehicles = sorted(self.vehicles_by_size_distance[size_distance], key=lambda v: v['topsis_score'], reverse=True)
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        solution = {v['vehicle_type']: 0 for v in vehicles}
        remaining_vehicles = max_vehicles
        total_topsis = sum(v['topsis_score'] for v in vehicles)
        
        for vehicle in vehicles:
            proportion = vehicle['topsis_score'] / total_topsis if total_topsis > 0 else 1 / len(vehicles)
            allocated_vehicles = int(proportion * max_vehicles)
            no_of_km = vehicle['demand'] / allocated_vehicles if allocated_vehicles > 0 else 0
            emissions_per_vehicle = vehicle['carbon_emissions_per_km'] * no_of_km
            
            if emissions_per_vehicle > 0:
                max_allowed_vehicles = min(allocated_vehicles, remaining_carbon // emissions_per_vehicle)
            else:
                max_allowed_vehicles = allocated_vehicles
            
            solution[vehicle['vehicle_type']] = int(max_allowed_vehicles)
            remaining_vehicles -= int(max_allowed_vehicles)
            remaining_carbon -= int(max_allowed_vehicles * emissions_per_vehicle)
            
            if remaining_carbon <= 0:
                break
        
        return solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        remaining_carbon = self.CARBON_LIMIT
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance, remaining_carbon)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                    if v['vehicle_type'] == vehicle_type)
                    no_of_km = vehicle_data['demand'] / num_vehicles if num_vehicles > 0 else 0
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions
                    remaining_carbon -= emissions
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['yearly_range']
                    })
        
        results_df = pd.DataFrame(results)
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = FleetOptimizer(data_df)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


Total Carbon Emissions: 11558344.620736381 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     LNG         357652.20          LNG   
1   Size S1, Distance D1     BEV              0.00  Electricity   
2   Size S1, Distance D2  Diesel        1765717.77          B20   
3   Size S1, Distance D2     LNG        1068657.04          LNG   
4   Size S1, Distance D3  Diesel        2238179.41          B20   
5   Size S1, Distance D3     LNG        1354602.77          LNG   
6   Size S2, Distance D1     LNG         409046.58          LNG   
7   Size S2, Distance D1  Diesel         679492.18          B20   
8   Size S2, Distance D1     BEV              0.00  Electricity   
9   Size S2, Distance D2     LNG         568238.42          LNG   
10  Size S2, Distance D3     LNG         319617.78          LNG   
11  Size S3, Distance D1     LNG         894747.21          LNG   
12  Size S3, Distance D1  Diesel        1491601.43          B20   
1

In [1]:
import pandas as pd
import random
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class GA_FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame, population_size=50, generations=100, mutation_rate=0.1):
        self.data = data
        self.vehicles_by_group = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        
    def _group_vehicles(self):
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self):
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_group.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def initialize_population(self, group):
        # Each individual is a dict mapping vehicle_type -> count
        vehicles = self.vehicles_by_group[group]
        max_vehicles = self.max_vehicles_by_group[group]
        population = []
        for _ in range(self.population_size):
            individual = {}
            remaining = max_vehicles
            for v in vehicles:
                # Randomly allocate vehicles while ensuring total does not exceed max
                count = random.randint(0, remaining)
                individual[v['vehicle_type']] = count
                remaining -= count
            population.append(individual)
        return population

    def fitness(self, individual, group):
        """
        Fitness function that calculates total carbon emissions.
        Penalize if demand is not met or if carbon limit is exceeded.
        Lower fitness is better.
        """
        vehicles = self.vehicles_by_group[group]
        total_emissions = 0
        demand = vehicles[0]['demand']
        total_allocated = sum(individual.values())
        penalty = 0
        
        # Penalize if fleet size is not enough to meet the demand (using max yearly range)
        max_yearly_range = max(v['yearly_range'] for v in vehicles)
        if total_allocated * max_yearly_range < demand:
            penalty += 1e8  # large penalty
        
        for v in vehicles:
            count = individual.get(v['vehicle_type'], 0)
            if count > 0:
                # calculate km per vehicle
                km_per_vehicle = v['demand'] / count
                emissions = count * v['carbon_emissions_per_km'] * km_per_vehicle
                total_emissions += emissions
                
        # If emissions exceed carbon limit, add penalty proportional to the excess.
        if total_emissions > self.CARBON_LIMIT:
            penalty += (total_emissions - self.CARBON_LIMIT) * 10
        
        return total_emissions + penalty

    def selection(self, population, group):
        # Tournament selection: choose two individuals and pick the one with lower fitness.
        selected = []
        for _ in range(len(population)):
            a, b = random.sample(population, 2)
            selected.append(a if self.fitness(a, group) < self.fitness(b, group) else b)
        return selected

    def crossover(self, parent1, parent2, group):
        # Single-point crossover for dictionary representation
        child = {}
        vehicle_types = list(parent1.keys())
        crossover_point = random.randint(1, len(vehicle_types) - 1)
        for i, vtype in enumerate(vehicle_types):
            if i < crossover_point:
                child[vtype] = parent1[vtype]
            else:
                child[vtype] = parent2[vtype]
        # Ensure that total allocated vehicles do not exceed the maximum
        max_vehicles = self.max_vehicles_by_group[group]
        total = sum(child.values())
        if total > max_vehicles:
            scale = max_vehicles / total
            for vtype in child:
                child[vtype] = int(child[vtype] * scale)
        return child

    def mutate(self, individual, group):
        # Mutate by randomly increasing or decreasing the count for one vehicle type.
        vehicle_types = list(individual.keys())
        vtype = random.choice(vehicle_types)
        change = random.choice([-1, 1])
        individual[vtype] = max(0, individual[vtype] + change)
        # Correct if total exceeds max vehicles
        max_vehicles = self.max_vehicles_by_group[group]
        total = sum(individual.values())
        if total > max_vehicles:
            excess = total - max_vehicles
            individual[vtype] = max(0, individual[vtype] - excess)
        return individual

    def evolve_population(self, population, group):
        # Selection
        selected = self.selection(population, group)
        next_generation = []
        # Crossover: pair up individuals
        for i in range(0, len(selected) - 1, 2):
            parent1 = selected[i]
            parent2 = selected[i+1]
            child = self.crossover(parent1, parent2, group)
            next_generation.append(child)
        # Mutation
        for i in range(len(next_generation)):
            if random.random() < self.mutation_rate:
                next_generation[i] = self.mutate(next_generation[i], group)
        # Fill up generation if needed
        while len(next_generation) < self.population_size:
            next_generation.append(random.choice(selected))
        return next_generation

    def optimize_group(self, group):
        population = self.initialize_population(group)
        best_individual = None
        best_fitness = float('inf')
        for gen in range(self.generations):
            population = self.evolve_population(population, group)
            for individual in population:
                fit = self.fitness(individual, group)
                if fit < best_fitness:
                    best_fitness = fit
                    best_individual = individual
        return best_individual

    def get_optimized_results(self):
        results = []
        total_carbon_emissions = 0
        for group in self.vehicles_by_group.keys():
            best_solution = self.optimize_group(group)
            vehicles = self.vehicles_by_group[group]
            max_vehicles = self.max_vehicles_by_group[group]
            for v in vehicles:
                num_vehicles = best_solution.get(v['vehicle_type'], 0)
                if num_vehicles > 0:
                    no_of_km = v['demand'] / num_vehicles
                    emissions = num_vehicles * v['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions
                    results.append({
                        "Allocation": f"Size {group[0]}, Distance {group[1]}",
                        "Vehicle": v['vehicle_type'],
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": v['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": v['demand'],
                        "Yearly Range": v['yearly_range']
                    })
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        results_df = pd.DataFrame(results)
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = GA_FleetOptimizer(data_df, population_size=50, generations=100, mutation_rate=0.1)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)


Total Carbon Emissions: 1197363.0522781448 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D4  Diesel         281685.36          B20   
2   Size S1, Distance D4  Diesel          45055.91          HVO   
3   Size S2, Distance D1     BEV              0.00  Electricity   
4   Size S2, Distance D4     LNG          54916.59          LNG   
5   Size S2, Distance D4     LNG           8457.65       BioLNG   
6   Size S3, Distance D1     BEV              0.00  Electricity   
7   Size S3, Distance D4  Diesel         140333.05          B20   
8   Size S3, Distance D4  Diesel          22446.44          HVO   
9   Size S4, Distance D1     BEV              0.00  Electricity   
10  Size S4, Distance D2  Diesel         505781.29          B20   
11  Size S4, Distance D2  Diesel          80900.32          HVO   
12  Size S4, Distance D3     LNG          49447.12          LNG   
1

In [3]:
import pandas as pd
import random
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class GA_FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame, population_size=50, generations=100, mutation_rate=0.1):
        self.data = data
        self.vehicles_by_group = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate
        
    def _group_vehicles(self):
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self):
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_group.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            # maximum number of vehicles needed so that demand can be met
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def initialize_population(self, group):
        """
        Create a random initial population where each individual's total 
        vehicle count <= max_vehicles_by_group[group].
        """
        vehicles = self.vehicles_by_group[group]
        max_vehicles = self.max_vehicles_by_group[group]
        population = []
        
        for _ in range(self.population_size):
            individual = {}
            remaining = max_vehicles
            
            # We randomly distribute 'remaining' across different vehicle types
            for v in vehicles:
                # at each step, choose a random number up to 'remaining'
                count = random.randint(0, remaining)
                individual[v['vehicle_type']] = count
                remaining -= count
            population.append(individual)
        
        return population

    def fitness(self, individual, group):
        """
        Fitness function that calculates total carbon emissions.
        Penalize if demand is not met or if carbon limit is exceeded.
        Lower fitness is better.
        """
        vehicles = self.vehicles_by_group[group]
        total_emissions = 0
        demand = vehicles[0]['demand']
        
        # Sum up the allocated vehicles
        total_allocated = sum(individual.values())
        penalty = 0
        
        # Check if the demand is met. The maximum range is from the best vehicle in that group
        # but here, we must consider each vehicle individually:
        #   demand met if sum( count_i * yearly_range_i ) >= demand
        total_range_capacity = 0
        for v in vehicles:
            count = individual.get(v['vehicle_type'], 0)
            total_range_capacity += count * v['yearly_range']
        
        if total_range_capacity < demand:
            # large penalty for not meeting demand
            penalty += 1e8
        
        # Calculate total carbon emissions
        # One approach: assume each vehicle runs proportionally to its share of total range capacity
        # A simpler approach: if we meet demand, we can compute how the demand might be split
        # among the allocated vehicles. For simplicity, let's do what you had: 
        # distribute the demand among the vehicles in proportion to their counts
        # or just assume each type covers a fraction of the demand based on how many vehicles you have
        # A straightforward approach:
        #    - If you have n vehicles of a type, each is responsible for demand / n of that type
        #    - But we need to ensure the sum of (count_i * yearly_range_i) >= demand
        # For each vehicle type:
        for v in vehicles:
            count = individual.get(v['vehicle_type'], 0)
            if count > 0:
                # Suppose each vehicle of this type travels v['demand'] / count 
                # BUT 'v['demand']' is the total group demand (the same for all vehicles).
                # A simpler approximation:
                km_per_vehicle = v['demand'] / total_allocated  # spread total demand across all vehicles equally
                # carbon emissions
                emissions = count * v['carbon_emissions_per_km'] * km_per_vehicle
                total_emissions += emissions
                
        # If emissions exceed carbon limit, add penalty
        if total_emissions > self.CARBON_LIMIT:
            penalty += (total_emissions - self.CARBON_LIMIT) * 10
        
        return total_emissions + penalty

    def selection(self, population, group):
        """
        Tournament selection: pick two individuals at random, 
        keep the one with the lower (better) fitness.
        """
        selected = []
        for _ in range(len(population)):
            a, b = random.sample(population, 2)
            selected.append(a if self.fitness(a, group) < self.fitness(b, group) else b)
        return selected

    def crossover(self, parent1, parent2, group):
        """
        Single-point crossover for dict representation.
        Then ensure total does not exceed max_vehicles.
        """
        vehicle_types = list(parent1.keys())
        crossover_point = random.randint(1, len(vehicle_types) - 1)
        
        child = {}
        for i, vtype in enumerate(vehicle_types):
            if i < crossover_point:
                child[vtype] = parent1[vtype]
            else:
                child[vtype] = parent2[vtype]
        
        # Fix the total if it exceeds max
        child = self._enforce_max_vehicles(child, group)
        return child

    def mutate(self, individual, group):
        """
        Randomly pick a vehicle type and add/subtract 1 from its count,
        while respecting the 0 and max constraints.
        """
        vehicle_types = list(individual.keys())
        vtype = random.choice(vehicle_types)
        
        if random.random() < 0.5:
            # Decrease by 1 if possible
            if individual[vtype] > 0:
                individual[vtype] -= 1
        else:
            # Increase by 1 if possible
            individual[vtype] += 1
        
        # Enforce the total does not exceed max
        individual = self._enforce_max_vehicles(individual, group)
        
        return individual

    def _enforce_max_vehicles(self, individual, group):
        """
        If total vehicles exceed max_vehicles_by_group[group], reduce proportionally.
        """
        max_vehicles = self.max_vehicles_by_group[group]
        total = sum(individual.values())
        if total > max_vehicles:
            # Scale down counts proportionally
            scale_factor = max_vehicles / total
            for k in individual.keys():
                individual[k] = int(individual[k] * scale_factor)
            
            # Because of int rounding, we might still exceed by 1 or so. Let's fix any leftover.
            overflow = sum(individual.values()) - max_vehicles
            while overflow > 0:
                # randomly reduce some vehicle by 1
                k = random.choice(list(individual.keys()))
                if individual[k] > 0:
                    individual[k] -= 1
                    overflow -= 1
        
        # Also ensure no negative values
        for k in individual:
            if individual[k] < 0:
                individual[k] = 0
        
        return individual

    def evolve_population(self, population, group):
        """
        1. Selection
        2. Crossover
        3. Mutation
        """
        selected = self.selection(population, group)
        next_generation = []
        
        # Crossover in pairs
        for i in range(0, len(selected) - 1, 2):
            parent1 = selected[i]
            parent2 = selected[i+1]
            child1 = self.crossover(parent1, parent2, group)
            child2 = self.crossover(parent2, parent1, group)
            next_generation.append(child1)
            next_generation.append(child2)
        
        # Mutation
        for i in range(len(next_generation)):
            if random.random() < self.mutation_rate:
                next_generation[i] = self.mutate(next_generation[i], group)
        
        # If next_generation is shorter, fill from selected
        while len(next_generation) < self.population_size:
            next_generation.append(random.choice(selected))
        
        return next_generation

    def optimize_group(self, group):
        population = self.initialize_population(group)
        best_individual = None
        best_fitness = float('inf')
        
        for _ in range(self.generations):
            population = self.evolve_population(population, group)
            
            # Track best in this generation
            for individual in population:
                fit = self.fitness(individual, group)
                if fit < best_fitness:
                    best_fitness = fit
                    best_individual = individual
        
        return best_individual

    def get_optimized_results(self):
        results = []
        total_carbon_emissions = 0
        
        for group in self.vehicles_by_group.keys():
            best_solution = self.optimize_group(group)
            vehicles = self.vehicles_by_group[group]
            max_vehicles = self.max_vehicles_by_group[group]
            
            # Calculate carbon emissions again for final reporting
            group_demand = vehicles[0]['demand']
            total_allocated = sum(best_solution.values())
            
            # Distribute demand across total_allocated vehicles
            # or each type gets a fraction of the total demand
            for v in vehicles:
                count = best_solution.get(v['vehicle_type'], 0)
                if count > 0:
                    # approximate each vehicle travels group_demand / total_allocated
                    km_per_vehicle = group_demand / total_allocated
                    emissions = count * v['carbon_emissions_per_km'] * km_per_vehicle
                    total_carbon_emissions += emissions
                    
                    results.append({
                        "Allocation": f"Size {group[0]}, Distance {group[1]}",
                        "Vehicle": v['vehicle_type'],
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": v['fuel'],
                        "no_of_vehicles": count,
                        "Max Vehicles": max_vehicles,
                        "Demand": v['demand'],
                        "Yearly Range": v['yearly_range']
                    })
        
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        results_df = pd.DataFrame(results)
        return results_df

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = GA_FleetOptimizer(data_df, population_size=50, generations=100, mutation_rate=0.1)
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)
    results_df.to_csv("optimized_fleet_Carbonmin1.csv", index=False)



Total Carbon Emissions: 6213144.258937818 (Limit: 11677957)
              Allocation Vehicle  Carbon Emissions         Fuel  \
0   Size S1, Distance D1     BEV              0.00  Electricity   
1   Size S1, Distance D2     LNG        1068657.04          LNG   
2   Size S1, Distance D2     LNG         162325.15       BioLNG   
3   Size S1, Distance D3     LNG        1354602.77          LNG   
4   Size S1, Distance D3     LNG         205759.27       BioLNG   
5   Size S1, Distance D4     LNG         170483.10          LNG   
6   Size S1, Distance D4     LNG          25895.77       BioLNG   
7   Size S2, Distance D1     BEV              0.00  Electricity   
8   Size S2, Distance D2     LNG         568238.42          LNG   
9   Size S2, Distance D2     LNG          87513.88       BioLNG   
10  Size S2, Distance D3     LNG         319617.78          LNG   
11  Size S2, Distance D3     LNG          49224.04       BioLNG   
12  Size S2, Distance D4     LNG          54916.59          LNG   
13

In [11]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

class FleetOptimizer:
    CARBON_LIMIT = 11677957  # Yearly carbon emission limit
    
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km'],
                'demand': row['Demand']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles
    
    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]
    
    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                solution[vehicle_type] += 1
                remaining_vehicles -= 1
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        
        return population
    
    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        if not self.is_valid_solution(solution, size_distance):
            return float('inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_carbon = sum(
            solution[v['vehicle_type']] * v['carbon_emissions_per_km'] * v['Yearly_range']
            for v in vehicles
        )
        
        penalty = max(0, total_carbon - self.CARBON_LIMIT) * 1000
        return total_carbon + penalty
    
    def mutate(self, solution: Dict, size_distance: Tuple, mutation_rate: float = 0.1) -> Dict:
        attempts = 0
        max_attempts = 10
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while attempts < max_attempts:
            mutated_solution = solution.copy()
            total_vehicles = sum(mutated_solution.values())
            
            for vehicle_type in mutated_solution.keys():
                if random.random() < mutation_rate:
                    change = random.choice([-1, 1])
                    if (change == 1 and total_vehicles < max_vehicles) or \
                       (change == -1 and mutated_solution[vehicle_type] > 0):
                        mutated_solution[vehicle_type] += change
                        total_vehicles += change
            
            if self.is_valid_solution(mutated_solution, size_distance):
                return mutated_solution
            
            attempts += 1
        
        return solution  
    
    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance)) for solution in population]
            fitness_scores.sort(key=lambda x: x[1])
            
            if fitness_scores[0][1] < best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            parents = [score[0] for score in fitness_scores[:len(population)//2]]
            next_generation = parents.copy()
            while len(next_generation) < len(population):
                parent1, parent2 = random.sample(parents, 2)
                child1, child2 = parent1.copy(), parent2.copy()
                child1 = self.mutate(child1, size_distance)
                child2 = self.mutate(child2, size_distance)
                next_generation.extend([child1, child2])
            
            population = next_generation[:len(population)]
        
        return best_solution
    
    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            max_vehicles = self.max_vehicles_by_group[size_distance]
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] if v['vehicle_type'] == vehicle_type)
                    no_of_km = vehicle_data['demand'] / num_vehicles
                    emissions = num_vehicles * vehicle_data['carbon_emissions_per_km'] * no_of_km
                    total_carbon_emissions += emissions
                    
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Carbon Emissions": round(emissions, 2),
                        "Fuel": "N/A",  
                        "no_of_vehicles": num_vehicles,
                        "Max Vehicles": max_vehicles,
                        "Demand": vehicle_data['demand'],
                        "Yearly Range": vehicle_data['Yearly_range']
                    })
        
        results_df = pd.DataFrame(results)
        print(f"Total Carbon Emissions: {total_carbon_emissions} (Limit: {self.CARBON_LIMIT})")
        return results_df


In [12]:
csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with your actual CSV file path
data_df = pd.read_csv(csv_path)
optimizer = FleetOptimizer(data_df)
results_df = optimizer.get_optimized_results()

# Print results
print(results_df)

# Save to CSV if needed
# results_df.to_csv("optimized_fleet_allocation.csv", index=False)


KeyError: 'Yearly_Range'

In [13]:
import pandas as pd
import random
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    df = pd.read_csv(csv_path)
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs_per_km',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_emissions_per_km': 'carbon_emissions_per_km'
    }
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    return df

class GA_FleetOptimizer:
    CARBON_LIMIT = 11677957  # Example carbon limit (adjust as needed)
    
    def __init__(self, 
                 data: pd.DataFrame, 
                 population_size=50, 
                 generations=100, 
                 mutation_rate=0.1):
        self.data = data
        self.vehicles_by_group = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
        self.population_size = population_size
        self.generations = generations
        self.mutation_rate = mutation_rate

    def _group_vehicles(self):
        """Group the vehicles by (Size, Distance)."""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'fuel_costs': row['Fuel_Costs_per_km'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups

    def _calculate_max_vehicles(self):
        """
        For each group, sum the no_of_vehicles values from the data.
        """
        max_vehicles = {}
        current_group = None
        current_sum = 0
        
        # Sort the data by Size and Distance_demand to group them together
        sorted_data = self.data.sort_values(['Size', 'Distance_demand'])
        
        for _, row in sorted_data.iterrows():
            group = (row['Size'], row['Distance_demand'])
            
            if group != current_group:
                if current_group is not None:
                    max_vehicles[current_group] = current_sum
                current_group = group
                current_sum = row['no_of_vehicles']
            else:
                current_sum += row['no_of_vehicles']
        
        # Don't forget the last group
        if current_group is not None:
            max_vehicles[current_group] = current_sum
            
        return max_vehicles

    def _initialize_population(self, group):
        """
        Initialize a population of dictionaries for one group, 
        where sum of all vehicles == max_vehicles_by_group[group].
        """
        vehicles = self.vehicles_by_group[group]
        max_veh = self.max_vehicles_by_group[group]
        population = []

        # If max_veh == 0 (e.g. zero demand?), skip or just fill with 0
        if max_veh == 0:
            return [{v['vehicle_type']: 0 for v in vehicles}] * self.population_size
        
        for _ in range(self.population_size):
            individual = {v['vehicle_type']: 0 for v in vehicles}
            remaining = max_veh
            vtypes = list(individual.keys())

            # Randomly distribute the 'max_veh' across the vehicle types
            for i in range(len(vtypes)):
                if i == len(vtypes) - 1:
                    # Last type takes whatever remains
                    individual[vtypes[i]] += remaining
                else:
                    alloc = random.randint(0, remaining)
                    individual[vtypes[i]] += alloc
                    remaining -= alloc
            
            population.append(individual)
        
        return population

    def _fitness(self, individual, group):
        """
        Fitness function for a single group's allocation:
          - sum of allocated vehicles == max_vehicles (already enforced in init/crossover/mutation).
          - penalize if demand is not met
          - penalize if carbon emissions exceed limit
          - otherwise, total emissions is the measure (lower is better).
        """
        vehicles = self.vehicles_by_group[group]
        demand = vehicles[0]['demand']
        
        # 1) Check if demand is met:
        #    sum( count_i * yearly_range_i ) >= demand
        total_range_capacity = 0
        for v in vehicles:
            count = individual[v['vehicle_type']]
            total_range_capacity += count * v['yearly_range']
        
        penalty = 0
        if total_range_capacity < demand:
            # large penalty
            penalty += 1e8

        # 2) Calculate total carbon emissions
        total_allocated = sum(individual.values())  # should be exactly max_veh
        total_emissions = 0.0
        
        # If total_allocated > 0, each vehicle covers demand / total_allocated km
        if total_allocated > 0:
            km_per_vehicle = demand / total_allocated
            for v in vehicles:
                count = individual[v['vehicle_type']]
                if count > 0:
                    emissions = count * v['carbon_emissions_per_km'] * km_per_vehicle
                    total_emissions += emissions

        # 3) If total emissions exceed carbon limit, penalize
        if total_emissions > self.CARBON_LIMIT:
            penalty += (total_emissions - self.CARBON_LIMIT) * 10

        return total_emissions + penalty

    def _selection(self, population, group):
        """
        Tournament selection: pick two random individuals, 
        keep the one with lower (better) fitness.
        """
        selected = []
        for _ in range(len(population)):
            a, b = random.sample(population, 2)
            fit_a = self._fitness(a, group)
            fit_b = self._fitness(b, group)
            selected.append(a if fit_a < fit_b else b)
        return selected

    def _crossover(self, parent1, parent2, group):
        """
        Single-point crossover. 
        Then adjust so sum of vehicles == max_vehicles exactly.
        """
        vtypes = list(parent1.keys())
        if len(vtypes) == 1:
            # Only one vehicle type in this group => child is the same
            return parent1.copy()
        
        crossover_point = random.randint(1, len(vtypes) - 1)
        child = {}
        for i, vt in enumerate(vtypes):
            if i < crossover_point:
                child[vt] = parent1[vt]
            else:
                child[vt] = parent2[vt]
        
        # Adjust sum to match exactly max_vehicles
        child = self._adjust_to_exact_sum(child, group)
        return child

    def _mutate(self, individual, group):
        """
        Mutation: pick two vehicle types (possibly the same),
        subtract 1 from one and add 1 to the other, 
        so the total sum remains the same.
        """
        if len(individual) == 1:
            # If there's only 1 vehicle type in this group, no real mutation
            return individual

        vtypes = list(individual.keys())
        vt1 = random.choice(vtypes)
        vt2 = random.choice(vtypes)
        
        # Only mutate if there's at least 1 vehicle in vt1 to move
        if individual[vt1] > 0:
            individual[vt1] -= 1
            individual[vt2] += 1
        
        # Make sure no negative
        if individual[vt1] < 0:
            individual[vt1] = 0
        
        # Then enforce exact sum
        individual = self._adjust_to_exact_sum(individual, group)
        return individual

    def _adjust_to_exact_sum(self, individual, group):
        """
        Make sure the sum of vehicles == max_vehicles for that group.
        If sum is too high, remove from random types; 
        if sum is too low, add to random types.
        """
        max_veh = self.max_vehicles_by_group[group]
        total = sum(individual.values())
        
        # If there's only 1 vehicle type, set it to max_veh
        if len(individual) == 1:
            vt = list(individual.keys())[0]
            individual[vt] = max_veh
            return individual
        
        # If total > max_veh, remove the excess
        while total > max_veh:
            vt = random.choice(list(individual.keys()))
            if individual[vt] > 0:
                individual[vt] -= 1
                total -= 1
        
        # If total < max_veh, add the difference
        while total < max_veh:
            vt = random.choice(list(individual.keys()))
            individual[vt] += 1
            total += 1

        # ensure no negative
        for vt in individual:
            if individual[vt] < 0:
                individual[vt] = 0
        
        return individual

    def _evolve_population(self, population, group):
        """
        GA steps: Selection -> Crossover -> Mutation
        """
        selected = self._selection(population, group)
        next_gen = []

        # Crossover in pairs
        for i in range(0, len(selected) - 1, 2):
            parent1 = selected[i]
            parent2 = selected[i+1]
            child1 = self._crossover(parent1, parent2, group)
            child2 = self._crossover(parent2, parent1, group)
            next_gen.append(child1)
            next_gen.append(child2)

        # Mutation
        for i in range(len(next_gen)):
            if random.random() < self.mutation_rate:
                next_gen[i] = self._mutate(next_gen[i], group)

        # If next_gen is smaller than population_size, fill up from selected
        while len(next_gen) < self.population_size:
            next_gen.append(random.choice(selected))

        return next_gen

    def optimize_group(self, group):
        """
        Run the GA for one group until convergence or generation limit.
        Return the best solution found.
        """
        population = self._initialize_population(group)
        best_individual = None
        best_fitness = float('inf')

        for _ in range(self.generations):
            population = self._evolve_population(population, group)
            # Check best in this generation
            for ind in population:
                fit = self._fitness(ind, group)
                if fit < best_fitness:
                    best_fitness = fit
                    best_individual = ind
        
        return best_individual

    def get_optimized_results(self):
        """
        For each group, run the GA to find an allocation 
        where sum(vehicle counts) == max_vehicles exactly.
        """
        results = []
        total_carbon_emissions = 0.0

        for group in self.vehicles_by_group.keys():
            best_solution = self.optimize_group(group)
            vehicles = self.vehicles_by_group[group]
            max_veh = self.max_vehicles_by_group[group]

            # Compute final carbon emissions for the chosen solution
            group_demand = vehicles[0]['demand']
            total_allocated = sum(best_solution.values())  # should be exactly max_veh
            group_emissions = 0.0

            if total_allocated > 0:
                km_per_vehicle = group_demand / total_allocated
                for v in vehicles:
                    count = best_solution[v['vehicle_type']]
                    if count > 0:
                        emissions = count * v['carbon_emissions_per_km'] * km_per_vehicle
                        group_emissions += emissions
                        
                        results.append({
                            "Allocation": f"Size {group[0]}, Distance {group[1]}",
                            "Vehicle": v['vehicle_type'],
                            "Carbon Emissions": round(emissions, 2),
                            "Fuel": v['fuel'],
                            "no_of_vehicles": count,
                            "Max Vehicles": max_veh,
                            "Demand": v['demand'],
                            "Yearly Range": v['yearly_range']
                        })
            
            total_carbon_emissions += group_emissions
        
        print(f"Total Carbon Emissions (sum of all groups): {total_carbon_emissions:.2f} "
              f"(Limit: {self.CARBON_LIMIT})")

        return pd.DataFrame(results)

def main(csv_path: str):
    data_df = load_and_preprocess_data(csv_path)
    optimizer = GA_FleetOptimizer(
        data_df, 
        population_size=50, 
        generations=100, 
        mutation_rate=0.1
    )
    optimized_results = optimizer.get_optimized_results()
    print(optimized_results)
    return optimized_results

if __name__ == "__main__":
    csv_path = "topsis_result/topsis_results_2023.csv"
    results_df = main(csv_path)

KeyError: 'no_of_vehicles'

In [5]:
def main():
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with the actual path to your dataset
    data = load_and_preprocess_data(csv_path)
    
    optimizer = FleetOptimizer(data)
    optimized_results = optimizer.get_optimized_results()
    
    print("Optimized Fleet Allocation:")
    print(optimized_results)

    # Save results to a CSV file
    # optimized_results.to_csv("optimized_fleet_allocation.csv", index=False)
    # print("Results saved to optimized_fleet_allocation.csv")

if __name__ == "__main__":
    main()


Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')
Optimized Fleet Allocation:
             Allocation Vehicle         Fuel  no_of_vehicles  \
0  Size S1, Distance D1     BEV  Electricity               1   
1  Size S2, Distance D1     BEV  Electricity               5   
2  Size S4, Distance D1     BEV  Electricity               1   

   Total Carbon Emissions  
0                     0.0  
1                     0.0  
2                     0.0  


In [6]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_er': 'carbon_emissions_per_km'
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)
    
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost': row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        return sum(solution.values()) <= self.max_vehicles_by_group[size_distance]

    def calculate_total_emissions(self, num_vehicles: int, vehicle: Dict) -> float:
        no_of_km = vehicle['demand'] / num_vehicles
        return num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        return population
    
    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_emissions += self.calculate_total_emissions(num_vehicles, vehicle)
        
        return -total_emissions if total_emissions <= 11677957 else float('-inf')
    
    def crossover(self, parent1: Dict, parent2: Dict) -> Dict:
        child = {}
        for key in parent1.keys():
            child[key] = parent1[key] if random.random() < 0.5 else parent2[key]
        return child

    def mutate(self, solution: Dict) -> Dict:
        mutation_rate = 0.1
        for key in solution.keys():
            if random.random() < mutation_rate:
                solution[key] = max(0, solution[key] + random.choice([-1, 1]))
        return solution
    
    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance))
                            for solution in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            new_population = []
            while len(new_population) < len(population):
                parent1, parent2 = random.choices(population, k=2)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                if self.is_valid_solution(child, size_distance):
                    new_population.append(child)
            population = new_population
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_emissions = self.calculate_total_emissions(num_vehicles, vehicle_data)
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Total Carbon Emissions": round(total_emissions, 2),
                    })
        return pd.DataFrame(results)


In [9]:
import pandas as pd
import random
from typing import List, Dict, Tuple
import numpy as np
import math

def load_and_preprocess_data(csv_path: str) -> pd.DataFrame:
    """Load and preprocess the fleet data from CSV file."""
    df = pd.read_csv(csv_path)
    
    # Rename columns properly
    column_mapping = {
        'Unnamed: 0': 'Index',
        'Distance_demand': 'Distance_demand',
        'Demand (km)': 'Demand',
        'Cost ($)': 'Cost',
        'Yearly range (km)': 'Yearly_Range',
        'insurance_cost': 'Insurance_Cost',
        'maintenance_cost': 'Maintenance_Cost',
        'fuel_costs_per_km': 'Fuel_Costs',
        'Fuel': 'Fuel',
        'Total_Cost': 'Total_Cost',
        'Topsis_Score': 'Topsis_Score',
        'carbon_er': 'carbon_emissions_per_km'
    }
    
    df.rename(columns=column_mapping, inplace=True, errors='ignore')
    print("Columns after renaming:", df.columns)
    
    return df

class FleetOptimizer:
    def __init__(self, data: pd.DataFrame):
        self.data = data
        self.vehicles_by_size_distance = self._group_vehicles()
        self.max_vehicles_by_group = self._calculate_max_vehicles()
    
    def _group_vehicles(self) -> Dict:
        """Group vehicles by (Size, Distance_demand) combination"""
        groups = {}
        for _, row in self.data.iterrows():
            key = (row['Size'], row['Distance_demand'])
            if key not in groups:
                groups[key] = []
            groups[key].append({
                'vehicle_type': row['Vehicle'],
                'yearly_range': row['Yearly_Range'],
                'topsis_score': row['Topsis_Score'],
                'insurance_cost': row['Insurance_Cost'],
                'maintenance_cost': row['Maintenance_Cost'],
                'Cost': row['Cost'],
                'fuel_costs_per_km': row['Fuel_Costs'],
                'fuel': row['Fuel'],
                'demand': row['Demand'],
                'carbon_emissions_per_km': row['carbon_emissions_per_km']
            })
        return groups
    
    def _calculate_max_vehicles(self) -> Dict:
        """Calculate maximum vehicles for each size-distance combination"""
        max_vehicles = {}
        for key, vehicles in self.vehicles_by_size_distance.items():
            demand = vehicles[0]['demand']  # All vehicles in the same group have the same demand
            max_yearly_range = max(v['yearly_range'] for v in vehicles)
            max_vehicles[key] = math.ceil(demand / max_yearly_range)
        return max_vehicles

    def is_valid_solution(self, solution: Dict, size_distance: Tuple) -> bool:
        return sum(solution.values()) == self.max_vehicles_by_group[size_distance]

    def calculate_total_emissions(self, num_vehicles: int, vehicle: Dict) -> float:
        if num_vehicles == 0:
            return 0  # Avoid division by zero
        no_of_km = vehicle['demand'] / num_vehicles
        return num_vehicles * vehicle['carbon_emissions_per_km'] * no_of_km

    def generate_initial_population(self, size_distance: Tuple, population_size: int = 50) -> List[Dict]:
        population = []
        vehicles = self.vehicles_by_size_distance[size_distance]
        max_vehicles = self.max_vehicles_by_group[size_distance]
        
        while len(population) < population_size:
            solution = {v['vehicle_type']: 0 for v in vehicles}
            remaining_vehicles = max_vehicles
            vehicle_types = list(solution.keys())
            while remaining_vehicles > 0 and vehicle_types:
                vehicle_type = random.choice(vehicle_types)
                if random.random() < 0.5:
                    solution[vehicle_type] += 1
                    remaining_vehicles -= 1
                else:
                    vehicle_types.remove(vehicle_type)
            
            if self.is_valid_solution(solution, size_distance):
                population.append(solution)
        return population
    
    def fitness_function(self, solution: Dict, size_distance: Tuple) -> float:
        if not self.is_valid_solution(solution, size_distance):
            return float('-inf')
        
        vehicles = self.vehicles_by_size_distance[size_distance]
        total_emissions = 0
        
        for vehicle in vehicles:
            num_vehicles = solution[vehicle['vehicle_type']]
            total_emissions += self.calculate_total_emissions(num_vehicles, vehicle)
        
        return -total_emissions
    
    def crossover(self, parent1: Dict, parent2: Dict) -> Dict:
        child = {}
        for key in parent1.keys():
            child[key] = parent1[key] if random.random() < 0.5 else parent2[key]
        return child

    def mutate(self, solution: Dict) -> Dict:
        mutation_rate = 0.1
        for key in solution.keys():
            if random.random() < mutation_rate:
                solution[key] = max(0, solution[key] + random.choice([-1, 1]))
        return solution
    
    def optimize(self, size_distance: Tuple, generations: int = 100) -> Dict:
        population = self.generate_initial_population(size_distance)
        best_solution = None
        best_fitness = float('-inf')
        
        for _ in range(generations):
            fitness_scores = [(solution, self.fitness_function(solution, size_distance)) for solution in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if fitness_scores[0][1] > best_fitness:
                best_solution = fitness_scores[0][0]
                best_fitness = fitness_scores[0][1]
            
            new_population = []
            while len(new_population) < len(population):
                parent1, parent2 = random.choices(population, k=2)
                child = self.crossover(parent1, parent2)
                child = self.mutate(child)
                if self.is_valid_solution(child, size_distance):
                    new_population.append(child)
            population = new_population
        
        return best_solution

    def get_optimized_results(self) -> pd.DataFrame:
        results = []
        total_carbon_emissions = 0
        
        for size_distance in self.vehicles_by_size_distance.keys():
            best_solution = self.optimize(size_distance)
            
            for vehicle_type, num_vehicles in best_solution.items():
                if num_vehicles > 0:
                    vehicle_data = next(v for v in self.vehicles_by_size_distance[size_distance] 
                                     if v['vehicle_type'] == vehicle_type)
                    total_emissions = self.calculate_total_emissions(num_vehicles, vehicle_data)
                    total_carbon_emissions += total_emissions
                    results.append({
                        "Allocation": f"Size {size_distance[0]}, Distance {size_distance[1]}",
                        "Vehicle": vehicle_type,
                        "Fuel": vehicle_data['fuel'],
                        "no_of_vehicles": num_vehicles,
                        "Total Carbon Emissions": round(total_emissions, 2),
                    })
        
        print(f"Total Carbon Emissions for all allocations: {round(total_carbon_emissions, 2)}")
        return pd.DataFrame(results)

In [10]:
def main():
    csv_path = "topsis_result/topsis_results_2023.csv"  # Replace with the actual path to your dataset
    data = load_and_preprocess_data(csv_path)
    
    optimizer = FleetOptimizer(data)
    optimized_results = optimizer.get_optimized_results()
    
    print("Optimized Fleet Allocation:")
    print(optimized_results)

    # Save results to a CSV file
    # optimized_results.to_csv("optimized_fleet_allocation.csv", index=False)
    # print("Results saved to optimized_fleet_allocation.csv")

if __name__ == "__main__":
    main()


Columns after renaming: Index(['Allocation', 'Operating Year', 'Size', 'Distance_demand', 'Demand',
       'ID', 'Vehicle', 'Available Year', 'Cost', 'Yearly_Range',
       'Distance_vehicle', 'Fuel', 'carbon_emissions_per_km', 'Insurance_Cost',
       'Maintenance_Cost', 'Fuel_Costs', 'Total_Cost', 'Topsis_Score', 'Rank',
       'Distance'],
      dtype='object')


KeyboardInterrupt: 